In [13]:
! bzip2 -dk 'data/simplewiki-20190220-pages-articles-multistream.xml.bz2'

bzip2: Output file data/simplewiki-20190220-pages-articles-multistream.xml already exists.


In [3]:
from bs4 import BeautifulSoup

In [4]:
def load_pages(file_name='data/simplewiki-20190220-pages-articles-multistream.xml.bz2.1'):
    file_name = 'data/enwiki-latest-pages-articles.xml'
    in_page = False
    page_lines = None

    with open (file_name) as f:
        for line in f:
            if line.strip() == '<page>':
                in_page = True
                page_lines = []
            if in_page: 
                page_lines.append(line)
            if line.strip() == '</page>':
                in_page = False
        yield ''.join(page_lines)

In [2]:
load_pages()

'  <page>\n    <title>File:Go North West logo.png</title>\n    <ns>6</ns>\n    <id>60822712</id>\n    <revision>\n      <id>898104976</id>\n      <timestamp>2019-05-21T12:14:11Z</timestamp>\n      <contributor>\n        <username>Nowell87</username>\n        <id>25998033</id>\n      </contributor>\n      <comment>Uploading a non-free logo using [[Wikipedia:File_Upload_Wizard|File Upload Wizard]]</comment>\n      <model>wikitext</model>\n      <format>text/x-wiki</format>\n      <text xml:space="preserve">==Summary==\n{{Non-free use rationale 2\n|Description = Go North West operator logo.\n|Source = https://www.gonorthwest.co.uk/\n|Article = Go North West\n|Purpose = to serve as the primary means of visual identification at the top of the article dedicated to the entity in question.\n|Replaceability = n.a.\n|Minimality = Used to demonstrate the company logo and branding.\n|Commercial = n.a.\n}}\n\n==Licensing==\n{{Non-free logo}}</text>\n      <sha1>t7pfdd76rup23l5ydub1drsf21p2m5q</sha1

In [9]:
def load_page_details(file_name='data/simplewiki-20190220-pages-articles-multistream.xml.bz2.1'):
    for page in load_pages(file_name):
        soup = BeautifulSoup(page)
        output = {}
        output['title'] = soup.select_one('title').text
        texts = soup.select('text')
        if len(texts) > 0:
            output['text'] = texts[0]
        yield output

In [10]:
next(iter(load_page_details()))

{'title': 'File:Go North West logo.png',
 'text': <text xml:space="preserve">==Summary==
 {{Non-free use rationale 2
 |Description = Go North West operator logo.
 |Source = https://www.gonorthwest.co.uk/
 |Article = Go North West
 |Purpose = to serve as the primary means of visual identification at the top of the article dedicated to the entity in question.
 |Replaceability = n.a.
 |Minimality = Used to demonstrate the company logo and branding.
 |Commercial = n.a.
 }}
 
 ==Licensing==
 {{Non-free logo}}</text>}

In [16]:
import pymongo

In [32]:
from pymongo import MongoClient

mc = pymongo.MongoClient()
db = mc['wikicache']
articles = db['articles']

In [35]:
articles.find_one()


{'_id': ObjectId('5d1fc2f4258b4b335c5b2093'),
 'title': 'Avans University of Applied Sciences',
 'text': "Avans University of Applied Sciences (Dutch: Avans Hogeschool) is a high-ranked Dutch vocational university. It is located in three cities: Breda, 's-Hertogenbosch, and Tilburg. The school has 30,000 students studying 40 courses in 18 institutes. There are 2,900 employees.\nAvans University of Applied Sciences itself was founded on January 1, 2004, as a union of Hogeschool 's-Hertogenbosch and Hogeschool Brabant in Tilburg, Breda, and Etten-Leur. Hogeschool Brabant itself was a union from 1988 of Hogeschool West-Brabant (Etten-Leur and Breda) and Hogeschool Midden-Brabant (Tilburg). The oldest branch of Avans University of Applied Sciences is the Kunstacademie in 's-Hertogenbosch, which was founded on October 1, 1812.\n\nRecognition\nAvans University of Applied Sciences is declared by an independent research from the Keuzegids HBO as the best major University of Applied Sciences in

In [26]:

def _connect_mongo(host, port, username, password, db):
    """Making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [27]:
read_mongo(db, articles, query={}, host='localhost', port=27017, username=None, password=None, no_id=True)

TypeError: name must be an instance of str